In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.utils import shuffle
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [10]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmark_casual.csv'
bikeshare_machine = set_data(file)

In [11]:
bikeshare_machine = bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,wind,rush_hour_1,holiday_1,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
145352,21.350,1.391010,0.294425,0.347277,1.079381,8.058349,57.5833,20.459254,0,0,...,0,0,0,0,1,0,0,0,1,0
124938,14.167,0.421029,0.328093,0.306432,0.604853,27.132500,57.8333,12.292557,0,0,...,1,0,0,0,0,0,0,1,0,0
16458,39.133,0.000000,0.250981,0.661659,0.661659,24.743349,43.4167,12.415904,1,0,...,0,0,0,0,1,0,0,1,0,0
169589,18.400,2.345214,0.133167,0.328533,0.580586,-2.947500,41.4583,12.374900,1,0,...,0,0,0,1,0,0,0,0,1,0
141049,57.767,2.176710,0.460965,0.455379,0.218430,15.663466,74.3043,9.522174,0,0,...,0,1,0,0,0,0,0,1,0,0


### Model 1 - Remove Some Variables that are correlated

In [12]:
# Leave workday, drop weekdays, leave season, drop month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['weekday_0', 'weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6', 
              'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9',
              'month_10','month_11','month_12','time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [13]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((152719, 275), (50907, 275), (152719,), (50907,))

In [14]:
# Fit model
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
pred1 = model1.predict(X1_test)
cvscores_model1 = cross_val_score(model1, X1_train, y1_train, cv=5)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [16]:
cvscores_model1, model1_r, adjustedr1, model1_mse, model1_rmse

(array([  1.58442358e-01,   1.52326309e-01,  -2.47345409e+14,
          1.51326758e-01,   1.49478048e-01]),
 0.15624521604588226,
 0.15166239987422092,
 283.48177223265003,
 16.836916945588644)

In [17]:
model1_sm = sm.OLS(y1_train, X1_train.astype(float)).fit()
model1_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     105.5
Date:                Sat, 16 Dec 2017   Prob (F-statistic):               0.00
Time:                        14:08:55   Log-Likelihood:            -6.4694e+05
No. Observations:              152719   AIC:                         1.294e+06
Df Residuals:                  152450   BIC:                         1.297e+06
Df Model:                         268                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             4.0148      0.055     72.824      0.000       3.907       4.123
x2             0.1237      1.041      0.119      0.905      -1.916       2.163
x3             0.8887      0.584      1.521      0.128      -0.256       2.034
x4             0.3577      0.462      0.774      0.439      -0.548       1.264
x5             0.0663      0.009      7.027      0.000       0.048       0.085
x6            -0.0025      0.004     -0.588      0.557      -0.011       0.006
x7             0.0078      0.010      0.798      0.425      -0.011       0.027
x8            -4.1177      0.209    -19.721      0.000      -4.527      -3.708
x9            -0.9988      0.239     -4.174      0.000      -1.468      -0.530
x10           -2.8511      0.091    -31.197      0.000      -3.030      -2.672
x11            1.8575      1.640      1.132      0.258      -1.358       5.073
x12           -3.5960      0.662     -5.435      0.000      -4.893      -2.299
x13            4.6531      0.333     13.962      0.000       4.000       5.306
x14           -6.6931      1.659     -4.035      0.000      -9.944      -3.442
x15           -5.4895      0.801     -6.851      0.000      -7.060      -3.919
x16            6.3211      1.344      4.702      0.000       3.686       8.956
x17           -1.6531      1.923     -0.860      0.390      -5.422       2.116
x18           -6.6370      0.703     -9.445      0.000      -8.014      -5.260
x19           -4.0836      0.761     -5.364      0.000      -5.576      -2.592
x20            1.2510      0.460      2.722      0.006       0.350       2.152
x21            5.5443      0.407     13.610      0.000       4.746       6.343
x22           -5.6225      0.870     -6.466      0.000      -7.327      -3.918
x23            3.3728      0.431      7.823      0.000       2.528       4.218
x24           -4.6348      0.722     -6.415      0.000      -6.051      -3.219
x25           -2.9697      0.476     -6.234      0.000      -3.903      -2.036
x26           -1.3020      0.417     -3.121      0.002      -2.120      -0.484
x27           -4.0076      0.437     -9.165      0.000      -4.865      -3.151
x28           -4.5224      1.116     -4.053      0.000      -6.710      -2.335
x29           -2.1778      1.532     -1.421      0.155      -5.181       0.825
x30            5.7914      1.010      5.734      0.000       3.812       7.771
x31           -1.8128      0.384     -4.720      0.000      -2.566      -1.060
x32           -7.1540      1.915     -3.736      0.000     -10.907      -3.401
x33           -4.1615      0.604     -6.892      0.000      -5.345      -2.978
x34           -2.3435      0.425     -5.511      0.000      -3.177      -1.510
x35            1.7892      0.548      3.265      0.001       0.715       2.863
x36           -2.2277      0.444     -5.013      0.000      -3.099      -1.357
x3

### Model 2 - Try reverse of Model 1

In [18]:
# Drop workday, leave weekdays, drop season, leave month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['work_day_1','season_1', 'season_2', 'season_3', 'season_4','time_diff']
X2 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y2 = bikeshare_machine['time_diff']

In [19]:
# Split the data into training and testing sets and check the shape
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.25)
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

((152719, 289), (50907, 289), (152719,), (50907,))

In [20]:
# Fit model
model2 = LinearRegression()
model2.fit(X2_train, y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
pred2 = model2.predict(X2_test)
cvscores_model2 = cross_val_score(model2, X2_train, y2_train, cv=5)
model2_r = model2.score(X2_test, y2_test)
model2_mse = mean_squared_error(y2_test, pred2)
model2_rmse = np.sqrt(model2_mse)
adjustedr2 = 1 - (1-model2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [22]:
cvscores_model2, model2_r, adjustedr2, model2_mse, model2_rmse

(array([ -4.71543384e+13,   1.62005545e-01,   1.54618498e-01,
         -1.98880462e+13,   1.61265650e-01]),
 0.15806506791659947,
 0.1532580031879095,
 280.0603381414939,
 16.735003380384896)

In [23]:
# Use statsmodel to check the statistical side of the model
model2_sm = sm.OLS(y2_train, X2_train.astype(float)).fit()
model2_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     102.5
Date:                Sat, 16 Dec 2017   Prob (F-statistic):               0.00
Time:                        14:09:17   Log-Likelihood:            -6.4697e+05
No. Observations:              152719   AIC:                         1.294e+06
Df Residuals:                  152437   BIC:                         1.297e+06
Df Model:                         281                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.9785      0.055     71.969      0.000       3.870       4.087
x2            -0.3880      1.095     -0.354      0.723      -2.534       1.758
x3             0.6701      0.568      1.180      0.238      -0.443       1.783
x4             0.6736      0.454      1.484      0.138      -0.216       1.563
x5             0.1164      0.014      8.396      0.000       0.089       0.144
x6            -0.0134      0.005     -2.790      0.005      -0.023      -0.004
x7            -0.0274      0.010     -2.695      0.007      -0.047      -0.007
x8            -4.0926      0.208    -19.653      0.000      -4.501      -3.684
x9             1.4983      0.265      5.662      0.000       0.980       2.017
x10            1.7164      1.689      1.016      0.309      -1.594       5.026
x11           -3.7168      0.689     -5.395      0.000      -5.067      -2.366
x12            4.5700      0.316     14.441      0.000       3.950       5.190
x13           -5.4518      1.686     -3.234      0.001      -8.755      -2.148
x14           -4.9209      0.807     -6.099      0.000      -6.502      -3.339
x15            6.4677      1.333      4.854      0.000       3.856       9.079
x16           -0.6650      1.937     -0.343      0.731      -4.462       3.132
x17           -6.0146      0.691     -8.704      0.000      -7.369      -4.660
x18           -3.5933      0.765     -4.698      0.000      -5.093      -2.094
x19            1.2871      0.454      2.834      0.005       0.397       2.177
x20            5.6380      0.399     14.122      0.000       4.856       6.421
x21           -5.0816      0.869     -5.848      0.000      -6.785      -3.378
x22            2.8532      0.416      6.855      0.000       2.037       3.669
x23           -4.5948      0.740     -6.207      0.000      -6.046      -3.144
x24           -2.3346      0.471     -4.962      0.000      -3.257      -1.412
x25           -1.0074      0.419     -2.407      0.016      -1.828      -0.187
x26           -3.7195      0.452     -8.237      0.000      -4.605      -2.834
x27           -4.2830      1.082     -3.958      0.000      -6.404      -2.162
x28           -1.1484      1.455     -0.789      0.430      -4.001       1.704
x29            6.2023      1.025      6.052      0.000       4.194       8.211
x30           -1.4650      0.389     -3.767      0.000      -2.227      -0.703
x31           -6.0823      1.948     -3.123      0.002      -9.900      -2.265
x32           -3.8239      0.613     -6.235      0.000      -5.026      -2.622
x33           -2.3166      0.414     -5.594      0.000      -3.128      -1.505
x34            1.5869      0.535      2.967      0.003       0.539       2.635
x35           -2.1300      0.435     -4.901      0.000      -2.982      -1.278
x36           -0.3932      1.031     -0.382      0.703      -2.413       1.627
x3

### Model 3 - All variables

In [24]:
# Keep all but time_diff
X3 = np.matrix(bikeshare_machine.drop('time_diff', 1))
y3 = bikeshare_machine['time_diff']

In [25]:
# Split the data into training and testing sets and check the shape
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.25)
X3_train.shape, X3_test.shape, y3_train.shape, y3_test.shape

((152719, 294), (50907, 294), (152719,), (50907,))

In [26]:
# Fit model
model3 = LinearRegression()
model3.fit(X3_train, y3_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [27]:
pred3 = model3.predict(X3_test)
cvscores_model3 = cross_val_score(model3, X3_train, y3_train, cv=5)
model3_r = model3.score(X3_test, y3_test)
model3_mse = mean_squared_error(y3_test, pred3)
model3_rmse = np.sqrt(model3_mse)
adjustedr3 = 1 - (1-model3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [28]:
cvscores_model3, model3_r, adjustedr3, model3_mse, model3_rmse

(array([  1.56268550e-01,  -1.97788069e+13,  -1.26791042e+11,
          1.56200985e-01,   1.54054087e-01]),
 0.15351783611850578,
 0.14860070665946123,
 279.25809201133882,
 16.711017084885611)

In [29]:
# Use statsmodel to check the statistical side of the model
model3_sm = sm.OLS(y3_train, X3_train.astype(float)).fit()
model3_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     102.6
Date:                Sat, 16 Dec 2017   Prob (F-statistic):               0.00
Time:                        14:09:43   Log-Likelihood:            -6.4703e+05
No. Observations:              152719   AIC:                         1.295e+06
Df Residuals:                  152434   BIC:                         1.297e+06
Df Model:                         284                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.9601      0.055     71.622      0.000       3.852       4.068
x2            -0.7234      1.011     -0.716      0.474      -2.704       1.257
x3             1.3772      0.532      2.590      0.010       0.335       2.419
x4             0.1192      0.471      0.253      0.800      -0.804       1.042
x5             0.1115      0.014      8.037      0.000       0.084       0.139
x6            -0.0110      0.005     -2.295      0.022      -0.020      -0.002
x7            -0.0234      0.010     -2.284      0.022      -0.043      -0.003
x8            -4.0633      0.208    -19.528      0.000      -4.471      -3.655
x9             3.4894      0.227     15.393      0.000       3.045       3.934
x10            2.1587      0.154     14.039      0.000       1.857       2.460
x11           -0.4983      1.579     -0.316      0.752      -3.592       2.596
x12           -3.9922      0.651     -6.128      0.000      -5.269      -2.715
x13            4.7048      0.314     14.998      0.000       4.090       5.320
x14           -6.4975      1.659     -3.917      0.000      -9.748      -3.247
x15           -5.7987      0.771     -7.520      0.000      -7.310      -4.287
x16            3.7976      1.341      2.833      0.005       1.170       6.425
x17           -2.0461      1.901     -1.077      0.282      -5.771       1.679
x18           -6.4465      0.671     -9.609      0.000      -7.761      -5.132
x19           -3.7993      0.738     -5.149      0.000      -5.246      -2.353
x20            1.5220      0.446      3.414      0.001       0.648       2.396
x21            5.6520      0.400     14.137      0.000       4.868       6.436
x22           -5.6901      0.858     -6.634      0.000      -7.371      -4.009
x23            2.6864      0.414      6.493      0.000       1.876       3.497
x24           -5.4649      0.706     -7.744      0.000      -6.848      -4.082
x25           -2.6423      0.467     -5.652      0.000      -3.559      -1.726
x26           -1.2407      0.407     -3.046      0.002      -2.039      -0.442
x27           -4.0621      0.440     -9.235      0.000      -4.924      -3.200
x28           -4.4072      1.050     -4.198      0.000      -6.465      -2.349
x29           -1.5618      1.411     -1.107      0.268      -4.327       1.204
x30            5.7847      0.982      5.893      0.000       3.861       7.709
x31           -1.5881      0.380     -4.174      0.000      -2.334      -0.842
x32           -7.2384      1.954     -3.704      0.000     -11.069      -3.408
x33           -3.9066      0.591     -6.605      0.000      -5.066      -2.747
x34           -2.6114      0.411     -6.348      0.000      -3.418      -1.805
x35            1.2162      0.529      2.300      0.021       0.180       2.253
x36           -1.6300      0.430     -3.795      0.000      -2.472      -0.788
x3

### Model 4 - Ridge with Round 1

In [30]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge1 = Ridge(fit_intercept=True)
ridge1_gs = GridSearchCV(ridge1, param_grid, cv=5)
ridge1_gs.fit(X1_train, y1_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [31]:
pred4 = ridge1_gs.predict(X1_test)
ridge1_r = ridge1_gs.score(X1_test, y1_test)
ridge1_mse = mean_squared_error(y1_test, pred4)
ridge1_rmse = np.sqrt(ridge1_mse)
adjustedr4 = 1 - (1-ridge1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [32]:
ridge1_r, ridge1_mse, ridge1_rmse, adjustedr4

(0.15623735819469864,
 283.48441228592361,
 16.836995346139513,
 0.15165449934347197)

### Ridge with Round 2

In [33]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge2 = Ridge(fit_intercept=True)
ridge2_gs = GridSearchCV(ridge2, param_grid, cv=5)
ridge2_gs.fit(X2_train, y2_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [34]:
pred5 = ridge2_gs.predict(X2_test)
ridge2_r = ridge2_gs.score(X2_test, y2_test)
ridge2_mse = mean_squared_error(y2_test, pred5)
ridge2_rmse = np.sqrt(ridge2_mse)
adjustedr5 = 1 - (1-ridge2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [35]:
ridge2_r, ridge2_mse, ridge2_rmse, adjustedr5

(0.15806130677476604,
 280.06158924370328,
 16.735040760144663,
 0.15325422057167037)

### Ridge with all data

In [36]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge3 = Ridge(fit_intercept=True)
ridge3_gs = GridSearchCV(ridge3, param_grid, cv=5)
ridge3_gs.fit(X3_train, y3_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [37]:
pred6 = ridge3_gs.predict(X3_test)
ridge3_r = ridge3_gs.score(X3_test, y3_test)
ridge3_mse = mean_squared_error(y3_test, pred6)
ridge3_rmse = np.sqrt(ridge3_mse)
adjustedr6 = 1 - (1-ridge3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [38]:
ridge3_r, ridge3_mse, ridge3_rmse, adjustedr6

(0.15351768867213056,
 279.2581406545296,
 16.711018540308356,
 0.14860055835658492)

In [39]:
#bikeshare_machine.to_csv('machine_full.csv')